# Hyperparameter Tuning 과제

- Manual Search
- Random Search (Grid Search 참고)
- Bayesian Optimization using HyperOpt
<br>연습해보기!

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import load_iris
import warnings
warnings.filterwarnings('ignore')
import time
from functools import partial

In [2]:
iris = load_iris()

X = iris.data
y = iris.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

## 1. Manual Search
사이킷런 Logistic Regresstion https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

In [6]:
'''
maunal 1 
'''

manual1 = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
                             intercept_scaling=1, class_weight=None, random_state=None,
                             solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False,
                             n_jobs=None, l1_ratio=None)
    # 첨부한 링크에 있는 사이킷런 도큐먼트를 참고하여 원하는대로 하이퍼파라미터 값을 넣어주세요
          

manual1.fit(X_train,y_train)
y_pred = manual1.predict(X_test)

# 정확도를 계산해주세요



In [7]:
man1_acc = accuracy_score(y_test,y_pred)
man1_acc

1.0

In [14]:
'''
maunal 2 
'''
# 다양한 하이퍼파라미터 값을 사용하여 또 다른 manual search를 해보세요
# manual1 에 비해 정확도가 더 높아지면 더 좋습니다 !

manual2 = LogisticRegression(penalty='none', dual=False, tol=0.001, C=1.0, fit_intercept=True,
                             intercept_scaling=1, class_weight=None, random_state=10,
                             solver='lbfgs', max_iter=200, multi_class='multinomial', verbose=0, warm_start=False,
                             n_jobs=None, l1_ratio=None)
    # 첨부한 링크에 있는 사이킷런 도큐먼트를 참고하여 원하는대로 하이퍼파라미터 값을 넣어주세요


manual2.fit(X_train,y_train)
y_pred2 = manual2.predict(X_test)

# 정확도를 계산해주세요



In [16]:
man2_acc = accuracy_score(y_test,y_pred2)
man2_acc

0.9736842105263158

첫번째가 더 좋은 결과가 나왔다.

## 2. Grid Search

In [17]:
from sklearn.model_selection import GridSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 그리드 서치 호출
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
grid_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 4.289061069488525


## 3. Random Search
grid search 코드와 매우 유사하니 참고해서 빈칸 채워주세요 !

In [18]:
from sklearn.model_selection import RandomizedSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 랜덤 서치 호출
random_search = RandomizedSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)  

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()

# 모델 적합
random_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)


test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 2.07230544090271


그리드 서치와 랜덤 서치가 결과는 같은데, 랜덤 서치의 시간이 더 적게 걸렸다.

## 4. HyperOpt

In [19]:
!pip install hyperopt

In [24]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

space = {
    'penalty' : hp.choice('penalty', ['l2', 'none']), 
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'max_iter' : hp.uniform('max_iter', 100,1000)
}

def objective(space):
    hopt = LogisticRegression(penalty = space['penalty'], 
                              fit_intercept = space['fit_intercept'],
                              max_iter = space['max_iter']
                                 )
    accuracy = cross_val_score(hopt, X_train, y_train, cv=4).mean()
    return {'loss': -accuracy, 'status': STATUS_OK }
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20,
            trials= trials)
best

100%|██████████| 20/20 [00:01<00:00, 13.68it/s, best loss: -0.9553571428571429]


{'fit_intercept': 0, 'max_iter': 775.6529267139556, 'penalty': 1}

In [28]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

pen = {0: 'l2',1: 'none'}
incpt = {0: True, 1: False}

trained = LogisticRegression(penalty = pen[best['penalty']], 
                              fit_intercept = incpt[best['fit_intercept']],
                              max_iter = best['max_iter']
                            ).fit(X_train,y_train)

# 정확도 도출 부분까지 코드 짜시면 됩니다!
hopt_pf = trained.predict(X_test)
print(confusion_matrix(y_test,hopt_pf))
print(classification_report(y_test,hopt_pf))
hopt_acc = accuracy_score(y_test,hopt_pf)

[[16  0  0]
 [ 0 11  0]
 [ 0  1 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.92      1.00      0.96        11
           2       1.00      0.91      0.95        11

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



## 선택) Bayesian Optimization 사용


In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization